In [1]:
%cd ../..
%run cryptolytic/notebooks/init.ipynb
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
import cryptolytic.util as util
import cryptolytic.start as start
import cryptolytic.viz.plot as plot
import cryptolytic.data.sql as sql
import cryptolytic.data as d
from cryptolytic.util import *
import cryptolytic.data.historical as h
import cryptolytic.model as m

from matplotlib.pylab import rcParams
from IPython.core.display import HTML
from pandas.plotting import register_matplotlib_converters # to stop a warning message


ohclv = ['open', 'high', 'close', 'low', 'volume']
plt.style.use('ggplot')
rcParams['figure.figsize'] = 20,7
start.init()
register_matplotlib_converters()


# Make math readable
HTML("""
<style>
.MathJax {
    font-size: 2rem;
}
</style>""")

c:\Users\kyleh\Desktop\Repos\cryptolytic-ds
Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [2]:
df = d.get_df ({'start':'06-01-2019', 'period':300, 'trading_pair':'eth_btc', 'exchange_id':'bitfinex'}, n=15000)
# df2 = d.get_df({'start':'05-07-2019', 'period':300, 'trading_pair':'btc_usd', 'exchange_id':'hitbtc'}, n=15000)
# df3 = d.get_df({'start':'05-07-2019', 'period':300, 'trading_pair':'btc_usdt', 'exchange_id':'poloniex'}, n=15000)
# df2 = d.get_df({'start':'04-05-2019', 'period':300, 'trading_pair':'eth_btc', 'exchange_id':'hitbtc'}, n=2000)

c:\Users\kyleh\Desktop\Repos\cryptolytic-ds\cryptolytic\data\__init__.py:54: FutureWarning:

how in .resample() is deprecated
the new syntax is .resample(...)..apply(<func>)



In [3]:
# TODO upgrade tensorflow to 2
# import tensorflow_transform as tft
# tft.pca(subdf.iloc[0], 3)

In [4]:
c = df[['close', 'volume', 'diff', 'arb_signal']]
a_df = c.rolling(6).mean().bfill().rename(columns=lambda x: x+'_mean')
b_df = c.rolling(6).std().bfill().rename(columns=lambda x: x+'_std')
c_df = c.rolling(6).skew().bfill().rename(columns=lambda x: x+'_skew')
d_df = c.rolling(6).kurt().bfill().rename(columns=lambda x: x+'_kurt')
df = pd.concat([df, a_df, b_df, c_df, d_df], axis=1).dropna(axis=1)
df.head()

,api,exchange,trading_pair,timestamp,period,open,close,high,low,volume,...,diff_std,arb_signal_std,close_skew,volume_skew,diff_skew,arb_signal_skew,close_kurt,volume_kurt,diff_kurt,arb_signal_kurt
datetime,,,,,,,,,,,,,,,,,,,,,
2019-01-06 08:00:00,bitfinex,bitfinex,eth_btc,1546761600,300.0,0.039770,0.039770,0.039770,0.039770,53.916301,...,0.000037,0.033986,-0.568393,1.215298,0.485575,0.325941,-1.685222,0.418789,-0.756465,-0.86204
2019-01-06 08:05:00,bitfinex,bitfinex,eth_btc,1546761900,300.0,0.039737,0.039735,0.039748,0.039735,4.151015,...,0.000037,0.033986,-0.568393,1.215298,0.485575,0.325941,-1.685222,0.418789,-0.756465,-0.86204
2019-01-06 08:10:00,bitfinex,bitfinex,eth_btc,1546762200,300.0,0.039735,0.039760,0.039760,0.039735,85.063575,...,0.000037,0.033986,-0.568393,1.215298,0.485575,0.325941,-1.685222,0.418789,-0.756465,-0.86204
2019-01-06 08:15:00,bitfinex,bitfinex,eth_btc,1546762500,300.0,0.039762,0.039690,0.039772,0.039673,413.201494,...,0.000037,0.033986,-0.568393,1.215298,0.485575,0.325941,-1.685222,0.418789,-0.756465,-0.86204
2019-01-06 08:20:00,bitfinex,bitfinex,eth_btc,1546762800,300.0,0.039691,0.039755,0.039755,0.039691,78.903445,...,0.000037,0.033986,-0.568393,1.215298,0.485575,0.325941,-1.685222,0.418789,-0.756465,-0.86204


In [5]:
df_sub = df.drop(['timestamp', 'period', 'open', 'high', 'low', 'api', 'exchange', 'trading_pair'], axis=1)
df_sub.head(100).tail(5)

,close,volume,diff,avg,arb_diff,arb_signal,close_mean,volume_mean,diff_mean,arb_signal_mean,...,diff_std,arb_signal_std,close_skew,volume_skew,diff_skew,arb_signal_skew,close_kurt,volume_kurt,diff_kurt,arb_signal_kurt
datetime,,,,,,,,,,,,,,,,,,,,,
2019-01-06 15:55:00,0.039692,30.240127,0.000030,0.03966666500000000000,0.000025,0.063870,0.039715,42.870302,0.000061,0.005397,...,0.000060,0.079118,-1.520739,-0.044150,0.951284,0.527292,1.804947,-2.149039,-0.681623,-0.834049
2019-01-06 16:00:00,0.039632,21.617551,0.000060,0.03958875000000000000,0.000043,0.109248,0.039696,32.602967,0.000052,0.026435,...,0.000054,0.088235,-0.491736,0.540091,1.774396,-0.223149,-2.157445,-1.643676,3.683828,-2.119237
2019-01-06 16:05:00,0.039567,69.620738,0.000066,0.03955875000000000000,0.000008,0.020855,0.039668,32.765557,0.000058,0.042689,...,0.000053,0.073132,-0.223048,0.558711,1.389503,-0.571255,-1.278260,-1.593250,2.698318,-0.125454
2019-01-06 16:10:00,0.039610,29.638142,0.000070,0.03959525000000000000,0.000015,0.037252,0.039645,37.035044,0.000064,0.047175,...,0.000052,0.071560,0.502473,0.625635,1.019402,-0.864654,-0.219334,-1.286994,2.091711,0.824889
2019-01-06 16:15:00,0.039590,17.322009,0.000032,0.03959450000000000000,-0.000005,-0.011365,0.039621,29.910087,0.000043,0.024388,...,0.000027,0.062935,0.697565,1.780054,-0.693524,-0.372850,0.913374,3.656532,-0.638285,0.348180


In [6]:
# rcParams['figure.figsize'] = 20,80
# df.plot(subplots=True);

In [7]:
def normalize_df(df):
    df = df.copy()
    df = df._get_numeric_data()
    # normalize each point by dividing by the inital point and subtracting 1
    for col in df.columns: 
        df[col] = (df[col] / (df[col][0] + 1e-5)) - 1
    return df
    
def denormalize_results(close_preds):
    return (1 + close_preds) * df.close[0]

In [8]:
def multivariate_data(dataset, target, start_index, end_index, history_size,
                      target_size, step, single_step=False):
    data = []
    labels = []
    
    start_index = start_index + history_size
    if end_index is None:
        end_index = len(dataset) - target_size
    
    for i in range(start_index, end_index):
        indices = range(i-history_size, i, step)
        data.append(dataset[indices])
        
        if single_step:
            labels.append(target[i+target_size])
        else:
            labels.append(target[i:i+target_size])
    
    return np.array(data), np.array(labels)

In [9]:
TRAIN_SPLIT = 3000
# dataset = normalize_df(df._get_numeric_data()).values
dataset = normalize_df(df._get_numeric_data()).values
target = df.columns.get_loc('close') - 1
y = dataset[:, target]
past_history = 720
future_target = 5
STEP = 3


x_train, y_train = multivariate_data(dataset, y, 0,
                                                   TRAIN_SPLIT, past_history,
                                                   future_target, STEP,
                                                   single_step=False)
x_val, y_val = multivariate_data(dataset, y,
                                               TRAIN_SPLIT, None, past_history,
                                               future_target, STEP,
                                               single_step=False)

In [10]:
len(dataset)

15000

In [11]:
x_train.shape, y_train.shape

((2280, 240, 26), (2280, 5))

In [12]:
print ('Single window of past history : {}'.format(x_train[0].shape))
print ('\n Target temperature to predict : {}'.format(y_train[0].shape))

Single window of past history : (240, 26)

 Target temperature to predict : (5,)


In [13]:
x_train.shape[-2:]

(240, 26)

In [14]:
import tensorflow as tf
BUFFER_SIZE = 10_000
BATCH_SIZE = 256
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

val_data = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_data = val_data.batch(BATCH_SIZE).repeat()

In [15]:
import tensorflow as tf
import tensorflow.keras.layers as layers

def lstm_model(train):
    model = tf.keras.models.Sequential()
    model.add(layers.Dense(128, use_bias=False))
    model.add(layers.LayerNormalization())
    model.add(layers.PReLU())
    model.add(layers.Dropout(0.20))
    model.add(layers.LSTM(128, return_sequences=True))
    model.add(layers.LSTM(128, activation='relu'))
              
    model.add(layers.Dense(future_target))
    model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss='mse')
    return model

U transition Matrix
$h_t = \phi(W x_t + U h_{t-1})$

In [16]:
# single_step_history = single_step_model.fit(train_data, epochs=7,
#                                             steps_per_epoch=38,
#                                             shuffle=False)                                            
##                                             validation_data=(x_val_single, y_val_single))

model = lstm_model(x_train)

history = model.fit(train_data,
                    epochs=9, 
                    steps_per_epoch=38,
                    use_multiprocessing=True,
                    workers=4,
                    validation_data=val_data,
                    validation_steps=5) #(x_val, y_val))


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Train for 38 steps, validate for 5 steps
Epoch 1/9
38/38 [==============================] - 542s 14s/step - loss: 0.0048 - val_loss: 0.0030
Epoch 2/9
38/38 [==============================] - 861s 23s/step - loss: 9.1225e-04 - val_loss: 0.0021
Epoch 3/9
38/38 [==============================] - 1090s 29s/step - loss: 6.3807e-04 - val_loss: 0.0036
Epoch 4/9
24/38 [=================>............] - ETA: 7:01 - loss: 5.1388e-04

In [0]:
model.summary()

In [0]:
def multi_step_plot(history, true_future, prediction):
    plt.figure(figsize=(12, 6))
    num_in = create_time_steps(len(history))
    num_out = len(true_future)
    
    plt.plot(num_in, np.array(history[:, 1]), label='History')
    plt.plot(np.arange(num_out)/STEP, np.array(true_future), 'bo',
        label='True Future')
    if prediction.any():
        plt.plot(np.arange(num_out)/STEP, np.array(prediction), 'ro',
            label='Predicted Future')
    plt.legend(loc='upper left')
    plt.show()
    
def plot_train_history(history, title):
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    
    epochs = range(len(loss))
    
    plt.figure()
    
    plt.plot(epochs, loss, 'b', label='Training loss')
    plt.plot(epochs, val_loss, 'r', label='Validation loss')
    plt.title(title)
    plt.legend()
    
    plt.show()


    
plot_train_history(history, 'Multi Step Training and validation loss') 

In [0]:
y

In [0]:
preds = denormalize_results(model.predict(x_train))
actual = df.close

# Predictions on the train set

In [0]:
rcParams['figure.figsize'] = 20,3
plt.plot(np.arange(2000), d.denoise(actual[past_history:2000+past_history], 5), label='actual')
plt.plot(range(2000), d.denoise(preds[:, 0][:2000], 5), label='predicted');
plt.legend();

In [0]:
plt.title('All predictions')
plt.plot(range(2000), d.denoise(preds[:2000], 5));

# Predictions on validation set

In [0]:
val_preds = denormalize_results(model.predict(x_val))
# val_actual = df.close.iloc[TRAIN_SPLIT:]
val_actual = denormalize_results(y_val[:, 0])
plt.plot(np.arange(2000), d.denoise(val_actual[past_history:2000+past_history], 20), label='actual')
plt.plot(range(2000), d.denoise(val_preds[:, 0][:2000], 20), label='predicted');
plt.legend();